# Download landsat scenes over the San Juan Islands

This notebook demonstrates:
* Using the pystac client to retrieve images from microsofts cloud storage
* Saving those images to netcdf

Code adapted from: https://medium.com/@geonextgis/getting-started-with-microsoft-planetary-computer-stac-api-67cbebe96e5e

In [30]:
import pystac_client
import planetary_computer
import odc.stac
from pystac.extensions.eo import EOExtension as eo

In [31]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace
)

#opening a connection to the STAC API endpoint hosted by the Planetary Computer platform
#check website for more info


In [29]:
# FHL location on San Juans
bbox_of_interest = [-123.33711789011167, 48.4, -122.65866282644907, 48.93242519875351]

#for times_of_interest, can do a list of dates, or a range of dates:


times_of_interest = [
    "2020-07-05/2020-07-05",
    "2020-07-13/2020-07-13",
    "2020-07-21/2020-07-21",
    "2020-07-29/2020-07-29",
    "2020-08-06/2020-08-06",
    "2020-08-14/2020-08-14",
    "2020-08-22/2020-08-22",
    "2020-08-30/2020-08-30",
    "2020-09-07/2020-09-07",
    "2020-09-15/2020-09-15",
    "2020-09-23/2020-09-23",
    "2020-10-01/2020-10-01"
    ]



# times_of_interest = [
#     "2022-07-27/2022-07-27",
#     "2022-08-04/2022-08-04",
#     "2022-08-12/2022-08-12",
#     "2022-08-20/2022-08-20",
#     "2022-09-29/2022-09-29",
#     "2021-06-30/2021-07-01",
# ]
#these are all the good dates from summer2022, listed in word doc from jessica

# Finding matching scenes for bbox_of_interest and times_of_interest

In [26]:
#for a range of times:
items_list = []
   #goes through times in list above
    # Query the catalog
search = catalog.search(
    collections=["landsat-c2-l2", "landsat-c1-l2"],
    bbox=bbox_of_interest,
        datetime=times_of_interest,
        # #  Uncomment the two lines below to sort the image collection based on 'cloud_cover'
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
)

items = search.item_collection()
print(f"Returned {len(items)} Items")
items_list.append(items)

Exception: too many datetime components (max=2, actual=3): ['2022-08-20/2022-08-20', '2022-08-04/2022-08-04', '2022-07-03/2022-07-03']

In [32]:
#for a list of times

items_list = []
for time_stamp in times_of_interest:   #goes through times in list above
    # Query the catalog
    search = catalog.search(
        collections=["landsat-c2-l2", "landsat-c1-l2"],
        bbox=bbox_of_interest,
        datetime=time_stamp,
        # #  Uncomment the two lines below to sort the image collection based on 'cloud_cover'
        # query={"eo:cloud_cover": {"lt": 10}},
        # sortby=["eo:cloud_cover"]
    )

    items = search.item_collection()
    print(f"Returned {len(items)} Items")
    items_list.append(items)

Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items
Returned 1 Items


In [11]:
items_list[0]

# Downloading scenes + adding temperature data
landsat visits a location every 16 days, 
this cell goes through all the scenes found above, downloads them, and adds temperature_degC calculated from lwir / lwir11

In [33]:
# These values are retrieved from Page 12 in this handbook: 
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat8-9-Collection2-Level2-Science-Product-Guide-v6.pdf
lwir_scale = 0.00341802
lwir_offset = 149.0



for items in items_list:
    for selected_item in items:
        try:
            ds = odc.stac.stac_load(
                [selected_item], bbox=bbox_of_interest
            ).isel(time=0)
            if 'lwir11' in ds: #got from plot_downloaded_scns
                temperature = ds["lwir11"].astype("float")
                temperature *= lwir_scale
                temperature += lwir_offset
                celsius = temperature - 273.15
                ds['temperature_degC'] = celsius
            if 'lwir' in ds:
                temperature = ds["lwir"].astype("float")
                temperature *= lwir_scale
                temperature += lwir_offset
                celsius = temperature - 273.15
                ds['temperature_degC'] = celsius
            #do temp calcs + add
            fn = f"{str(ds.time.values)}.nc" 
            print(fn)
            ds.to_netcdf(fn)
        except Exception as e: 
            print(f"Failed on {selected_item} with error: {e}")

2020-07-05T18:31:52.785482000.nc
2020-07-13T19:01:17.887145000.nc
2020-07-21T18:30:54.112057000.nc
2020-07-29T19:01:22.439132000.nc
2020-08-06T18:29:54.122643000.nc
2020-08-14T19:01:26.426532000.nc
2020-08-22T18:28:52.743266000.nc
2020-08-30T19:01:34.768005000.nc
2020-09-07T18:27:50.030860000.nc
2020-09-15T19:01:40.966120000.nc
2020-09-23T18:26:46.234346000.nc
2020-10-01T19:01:44.969542000.nc


In [24]:
ds

<xarray.Dataset> Size: 149MB
Dimensions:           (y: 1974, x: 1675)
Coordinates:
  * y                 (y) float64 16kB 5.42e+06 5.42e+06 ... 5.361e+06 5.361e+06
  * x                 (x) float64 13kB 4.75e+05 4.751e+05 ... 5.253e+05
    spatial_ref       int32 4B 32610
    time              datetime64[ns] 8B 2019-08-03T19:07:41.114082
Data variables: (12/20)
    qa                (y, x) int16 7MB 213 213 213 213 ... -9999 -9999 -9999
    red               (y, x) uint16 7MB 7560 7557 7569 7580 7586 ... 0 0 0 0 0
    blue              (y, x) uint16 7MB 7362 7376 7399 7383 7360 ... 0 0 0 0 0
    drad              (y, x) int16 7MB 882 882 882 882 ... -9999 -9999 -9999
    emis              (y, x) int16 7MB 9880 9880 9880 9880 ... -9999 -9999 -9999
    emsd              (y, x) int16 7MB 0 0 0 0 0 ... -9999 -9999 -9999 -9999
    ...                ...
    swir22            (y, x) uint16 7MB 7829 7827 7862 7858 7844 ... 0 0 0 0 0
    coastal           (y, x) uint16 7MB 6735 6744 6770 6775 6784 ... 0 0 0 0 0
    qa_pixel          (y, x) uint16 7MB 21952 21952 21952 21952 ... 1 1 1 1
    qa_radsat         (y, x) uint16 7MB 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    qa_aerosol        (y, x) uint8 3MB 224 192 224 224 192 224 ... 1 1 1 1 1 1
    temperature_degC  (y, x) float64 26MB 17.82 17.82 17.83 ... -124.1 -124.1